# Dimension Reduction Techniques
+ Instead of using the original predictors, we transform them first and then fit our models. 
+ Usually transform variables so that there are less in number than the original set.
+ Then fit a least squares model using the transformed variables.
+ Let $Z_1, Z_2, \ldots, Z_M$ represent $M<p$ linear combinations of original $p$ predictors.
$$Z_m = \sum_{j=1}^p{\phi_{jm}X_j}$$
for some constants $\phi_{1m}, \phi_{2m}, \ldots, \phi_{pm},\; m=1, \ldots, M$.
+ We can then fit the linear regression model
$$y_i = \theta_0 + \sum_{m=1}^M{\theta_m z_{im}} + \epsilon_i, \quad i=1, \ldots, n$$
using least squares.
+ Notice that
$$ \sum_{m=1}^M{\theta_m z_{im}} = \sum_{m=1}^M{\theta_m}\sum_{j=1}^p{\phi_{jm}x_{ij}} = \sum_{j=1}^p{\sum_{m=1}^M{\theta_m\phi_{jm}x_{ij}}} = \sum_{j=1}^p{\beta_jx_{ij}} $$
where
$$\beta_j = \sum_{m=1}^M{\theta_m\phi_{jm}}$$

## Principal Component Analysis (PCA)
+ Here we describe its use as a dimension reduction technique for regression.

+ The first principal component is the direction where observations vary the most (Fig. 6.14). 
+ We want to capture as much information as we can in one single direction.
+ Which single direction captures as much information as possible? 
+ The direction where the variance is highest amongst the projected points.

+ The first principal component also minimizes the sum of squared perpendicular distances between point and line. 


In [ ]:
from IPython.display import Image
Image('images/pw48.png', width =700)

In [ ]:
Image('images/pw49.png', width =700)

+ The first principal component is
$$Z_1 = 0.839 \times (\tt{pop} - \overline{\tt{pop}}) + 0.544 \times (\tt{ad} - \overline{\tt{ad}}) $$
+ $\phi_{11} = 0.839$ and $\phi_{21} = 0.544$
+ $\phi_{11}^2 + \phi_{21}^2 = 1$
+ Since $n = 100$, $\tt{pop}$ and $\tt{ad}$ are vectors of length 100, and so is $Z_1$
$$z_{i1} = 0.839 \times (\tt{pop}_i - \overline{\tt{pop}}) + 0.544 \times (\tt{ad}_i - \overline{\tt{ad}}) $$
+ $z_i1$ are the **principal component scores** (Right Fig 6.15).
+ Each transformed first principal component can be thought as single number summaries of all that particular observation.
+ For this example, if $z_{i1}<0$, this indicates a city with below-average population size and below average ad spending. A positive score suggests the opposite.

In [ ]:
Image('images/pw50.png', width =700)

+ The second principal component must be uncorrelated to the first which makes it orthogonal (90 degrees in two dimensions) to the first. 
+ The second principal component is
$$Z_2 = 0.544 \times (\tt{pop} - \overline{\tt{pop}}) - 0.839 \times (\tt{ad} - \overline{\tt{ad}}) $$
+ The second PC will capture less information (less spread). 
+ Plotting each PC against each variable can show how much information is captured by each one.


In [ ]:
Image('images/pw51.png', width =700)

**Conclusion for the Example:**
There is little relationship between the second principal component and these two predictors, suggesting, one only needs the first principal component in order to accurately represent the pop and ad budgets.

## Principal Component Regression
1. Find first M principal components where M < p then fit with least squares. 
2. Choose M with cross validation. 
+ Usually, data is standardized by standard deviation first.



In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression, PLSSVD
from sklearn.metrics import mean_squared_error


df = pd.read_csv('data/hitters.csv').dropna().drop('Unnamed: 0', axis=1)
df.info()
dummies = pd.get_dummies(df[['League', 'Division', 'NewLeague']])

In [ ]:
df.head()

In [ ]:
y = df.Salary

# Drop the column with the independent variable (Salary), and columns for which we created dummy variables
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')

# Define the feature set X.
X = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)

In [ ]:
X.head()

Unfortunately `sklearn` does not have an implementation of PCA and regression combined,  so we'll have to do it ourselves.

We'll start by performing Principal Components Analysis (PCA), remember to scale the data:

In [ ]:
pca = PCA()
X_reduced = pca.fit_transform(scale(X))   ###scale the data

Print out the first few variables of the first few principal components:


In [ ]:
pd.DataFrame(pca.components_.T).loc[:4,:5]

Now we'll perform 10-fold cross-validation to see how it influences the MSE:

In [ ]:
# 10-fold CV, with shuffle
n = len(X_reduced)
kf_10 = model_selection.KFold(n_splits=10, shuffle=True, random_state=1)

regr = LinearRegression()
mse = []

# Calculate MSE with only the intercept (no principal components in regression)
score = -1*model_selection.cross_val_score(regr, np.ones((n,1)), y.ravel(), cv=kf_10, scoring='neg_mean_squared_error').mean()    
mse.append(score)

# Calculate MSE using CV for the 19 principle components, adding one component at the time.
for i in np.arange(1, 20):
    score = -1*model_selection.cross_val_score(regr, X_reduced[:,:i], y.ravel(), cv=kf_10, scoring='neg_mean_squared_error').mean()
    mse.append(score)
    
# Plot results    
plt.plot(mse, '-v')
plt.xlabel('Number of principal components in regression')
plt.ylabel('MSE')
plt.title('Salary')
plt.xlim(xmin=-1);

+ We see that the smallest cross-validation error occurs when $M = 18$ components are used. 
+ This is barely fewer than $M = 19$, which amounts to simply performing least squares, because when all of the components are used in PCR no dimension reduction occurs. 
+ However, from the plot we also see that the cross-validation error is roughly the same when only one component is included in the model. 
+ This suggests that a model that uses just a small number of components might suffice.

In [ ]:
## Amount of variance explained by adding each consecutive principal component:
np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

+ We can think of this as the amount of information about the predictors or the response that is captured using $M$ principal components. 
+ For example, setting $M = 1$ only captures 38.31% of all the variance, or information, in the predictors. 
+ In contrast, using $M = 6$ increases the value to 88.63%. If we were to use all $M = p = 19$ components, this would increase to 100%.


Now, perform PCA on the training data and evaluate its test set performance:


In [ ]:

pca2 = PCA()

# Split into training and test sets
X_train, X_test , y_train, y_test = model_selection.train_test_split(X, y, test_size=0.5, random_state=1)

# Scale the data
X_reduced_train = pca2.fit_transform(scale(X_train))
n = len(X_reduced_train)

# 10-fold CV, with shuffle
kf_10 = model_selection.KFold(n_splits=10, shuffle=True, random_state=1)

mse = []

# Calculate MSE with only the intercept (no principal components in regression)
score = -1*model_selection.cross_val_score(regr, np.ones((n,1)), y_train.ravel(), cv=kf_10, scoring='neg_mean_squared_error').mean()    
mse.append(score)

# Calculate MSE using CV for the 19 principle components, adding one component at the time.
for i in np.arange(1, 20):
    score = -1*model_selection.cross_val_score(regr, X_reduced_train[:,:i], y_train.ravel(), cv=kf_10, scoring='neg_mean_squared_error').mean()
    mse.append(score)

plt.plot(np.array(mse), '-v')
plt.xlabel('Number of principal components in regression')
plt.ylabel('MSE')
plt.title('Salary')
plt.xlim(xmin=-1);

Lowest cross-validation error occurs when  M=6  components are used.

Performance on the test data:


In [ ]:
X_reduced_test = pca2.transform(scale(X_test))[:,:7]

# Train regression model on training data 
regr = LinearRegression()
regr.fit(X_reduced_train[:,:7], y_train)

# Prediction with test data
pred = regr.predict(X_reduced_test)
mean_squared_error(y_test, pred)

## Partial Least Squares Regression
+ For PCR, the response does not determine the principal components, which means that the PCA is used in an unsupervised way. 
+ PLSR is a supervised alternative to PCR. 
+ PLSR generates new features as a linear combination of the old features and the response
$$Z_m = \sum_{j=1}^p{\phi_{jm}X_j}$$
$$y_i = \theta_0 + \sum_{m=1}^M{\theta_m z_{im}} + \epsilon_i, \quad i=1, \ldots, n$$
+ Computed by doing simple linear regression of $Y$ onto each predictor and setting that coefficient to the linear combination coefficient for transformed variable $Z_1$. 
+ So weights are higher for those variables with stronger relationships to response. 
+ $Z_2$ is computed by regressing all variables against the residuals of $Z_1$ being fit to the model. 
+ Do this iteratively (fit remaining residuals) to come up with $M$ PLS components. 
+ Then do least squares fit on all $M$ new dimensions. 
+ In practice PLSR does not do better than PCR or ridge regression.



In [ ]:
##Scikit-learn PLSRegression gives same results as the pls package in R when using method='oscorespls'. 
##However, the standard method used is 'kernelpls', which we'll use here.

n = len(X_train)

# 10-fold CV, with shuffle
kf_10 = model_selection.KFold(n_splits=10, shuffle=True, random_state=1)

mse = []

for i in np.arange(1, 20):
    pls = PLSRegression(n_components=i)
    score = model_selection.cross_val_score(pls, scale(X_train), y_train, cv=kf_10, scoring='neg_mean_squared_error').mean()
    mse.append(-score)

# Plot results
plt.plot(np.arange(1, 20), np.array(mse), '-v')
plt.xlabel('Number of principal components in regression')
plt.ylabel('MSE')
plt.title('Salary')
plt.xlim(xmin=-1)

In [ ]:
## The lowest cross-validation error occurs when only  M=2  partial least squares dimensions are used.

pls = PLSRegression(n_components=3)
pls.fit(scale(X_train), y_train)

mean_squared_error(y_test, pls.predict(scale(X_test)))

In [ ]:
pls.x_weights_

# High Dimensional Data
+ When speaking of high dimensional data, we generally mean data with many predictors, especially when p approaches or exceeds n. 
+ Generally it is better to have more predictors but if many of the predictors are not associated with the response then they can cause the actual signal to get diluted - a double edged sword these predictors.

# Exercises


## 6a
$(y_1 - \beta_1)^2 + \lambda\beta_1^2$

Problem allows me to choose $y_1$ and $\lambda$. I'll choose $y_1 = 5$ and $\lambda = 3$

In [ ]:
# plot this as a function of beta
#(5 - beta)^2 + 3beta^2
import numpy as np
beta = np.linspace(-10, 10, 1000)
y = 5
lam = 3
ridge = (y - beta)**2 + lam * beta**2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(beta, ridge)

In [ ]:
# min from plot
beta[np.argmin(ridge)]

In [ ]:
# min from 6.14
y / (1 + lam) # confirmed!

# 6b
do similar thing for lasso

In [ ]:
beta = np.linspace(-10, 10, 1000)
y = 5
lam = 3
lasso = (y - beta)**2 + lam * abs(beta)
plt.plot(beta, lasso)

In [ ]:
beta[np.argmin(lasso)]

In [ ]:
# min from 6.15
# since y > lambda / 2 minimum should be at y - lambda / 2
y - lam / 2 # confirmed!

# 8

In [ ]:
x = np.random.randn(100)
err = np.random.randn(100)

In [ ]:
beta0, beta1, beta2, beta3 = -5, 1, 4, 3
y = beta0 + beta1 * x + beta2 * x ** 2 + beta3 * x ** 3 + err

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from itertools import combinations

In [ ]:
from collections import OrderedDict

In [ ]:
OrderedDict({'b': 1, 'a':534})

In [ ]:
df = pd.DataFrame({'x1': x, 'x2': x ** 2, 'x3': x**3, 'x4': x**4,'x5': x**5,
                   'x6': x**6,'x7': x**7,'x8': x**8,'x9': x**9,'x9_10': x**10,
                   'y':y})

In [ ]:
df.head()

In [ ]:
lr = LinearRegression()

In [ ]:
X = df.iloc[:, :-1]
y = df['y']

In [ ]:
lr.fit(X, y)

In [ ]:
sigma2 = np.sum((lr.predict(X) - y) ** 2) / len(X)

In [ ]:
# best subset selection
n = len(X)
cp = []
bic = []
adj_r2 = []
for i in range(1, 11):
    current_cp = []
    current_bic = []
    current_adj_r2 = []
    for comb in combinations(range(10), i):
        X = df.iloc[:, comb]
        lr.fit(X, y)
        rss = np.sum((lr.predict(X) - y) ** 2)
        tss = np.sum((y - y.mean()) ** 2)
        d = len(comb)
        current_cp.append(1/n * (rss + 2 * d * sigma2))
        current_bic.append(1/n * (rss + np.log(n) * d * sigma2))
        current_adj_r2.append(1 - rss / (n - d - 1) * (n - 1) / tss)
        
    cp.append(min(current_cp))
    bic.append(min(current_bic))
    adj_r2.append(max(current_adj_r2))

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(range(1, 11), cp)
plt.plot(range(1, 11), bic)
plt.title("CP and BIC Best subset");

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(range(1, 11), adj_r2)
plt.title("Adjusted R^2");

In [ ]:
# all three agree on correct model!
np.argmin(cp), np.argmin(bic), np.argmax(adj_r2)

In [ ]:
cp

In [ ]:
# forward selection. Looks at Cp each step and stops if it can't beat old best
current_vars = []
best_cp = 10000000
prev_cp = best_cp
best_cp = 1000000
while best_cp < prev_cp:
    prev_cp = best_cp
    old_vars = current_vars.copy()
    for i in range(10):
        if i in current_vars:
            continue
        X = df.iloc[:, old_vars + [i]]
        lr.fit(X, y)
        rss = np.sum((lr.predict(X) - y) ** 2)
        d = len(old_vars) + 1
        cur_cp = 1/n * (rss + 2 * d * sigma2)
        if cur_cp < best_cp:
            current_vars = old_vars + [i]
            best_cp = cur_cp

In [ ]:
current_vars

In [ ]:
best_cp

In [ ]:
old_vars.

In [ ]:
# backward selection. Looks at Cp each step and stops if it can't beat old best
current_vars = list(range(10))
best_cp = 10000000
prev_cp = best_cp
best_cp = 1000000
while best_cp < prev_cp:
    prev_cp = best_cp
    old_vars = current_vars.copy()
    for i in range(10):
        if i not in current_vars:
            continue
        old_vars2 = old_vars.copy()
        old_vars2.remove(i)
        X = df.iloc[:, old_vars2]
        lr.fit(X, y)
        rss = np.sum((lr.predict(X) - y) ** 2)
        d = len(old_vars) + 1
        cur_cp = 1/n * (rss + 2 * d * sigma2)
        if cur_cp < best_cp:
            current_vars = old_vars2.copy()
            best_cp = cur_cp

In [ ]:
current_vars # same answer for backward selection

In [ ]:
X = df.iloc[:, :-1]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso

In [ ]:
X_stand = X / X.std()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_stand, y)

In [ ]:
alphas = np.linspace(.0001, .1, 1000)
errors = []
for alpha in alphas:
    ls = Lasso(alpha, max_iter=100000, tol=.0001)
    ls.fit(X_train, y_train)
    errors.append(np.mean((ls.predict(X_test) - y_test) ** 2))

In [ ]:
plt.plot(alphas, errors)

In [ ]:
np.argmin(errors)

In [ ]:
alphas[53]

In [ ]:
ls = Lasso(alpha=.0054, max_iter=100000, tol=.0001)
ls.fit(X_stand, y)

In [ ]:
ls.intercept_, ls.coef_

In [ ]:
# beta 3 was very far off
beta0, beta1, beta2, beta3

### f

In [ ]:
beta0_7 = 3
beta7 = -1

In [ ]:
y_7 = beta0_7 + beta7 * x ** 7 + err

In [ ]:
df_7 = pd.DataFrame({'x1': x, 'x2': x ** 2, 'x3': x**3, 'x4': x**4,'x5': x**5,
                   'x6': x**6,'x7': x**7,'x8': x**8,'x9': x**9,'x9_10': x**10,
                   'y':y_7})

In [ ]:
plt.scatter(X['x7'], y_7)

In [ ]:
# best subset selection
X = df_7.iloc[:, :-1]
n = len(X)
tss = np.sum((y_7 - y_7.mean()) ** 2)
lr.fit(X,  y_7)
sigma2 = np.sum((lr.predict(X) - y_7) ** 2) / len(X)
cp = []
bic = []
adj_r2 = []
for i in range(1, 11):
    current_cp = []
    current_bic = []
    current_adj_r2 = []
    for comb in combinations(range(10), i):
        X = df_7.iloc[:, comb]
        lr.fit(X, y_7)
        rss = np.sum((lr.predict(X) - y_7) ** 2)
        
        d = len(comb)
        current_cp.append(1/n * (rss + 2 * d * sigma2))
        current_bic.append(1/n * (rss + np.log(n) * d * sigma2))
        current_adj_r2.append(1 - rss / (n - d - 1) * (n - 1) / tss)
        
    cp.append(min(current_cp))
    bic.append(min(current_bic))
    adj_r2.append(max(current_adj_r2))

In [ ]:
# best model is with one predictor
plt.plot(range(10), cp)
bic.append(min(current_bic))

In [ ]:
# lasso
X = df_7.iloc[:, :-1]
X_stand = X / X.std()
X_train, X_test, y_train, y_test = train_test_split(X_stand, y_7)

In [ ]:
alphas = np.linspace(.001, 50, 100)
errors = []
ls = Lasso(alpha, max_iter=1000000000, tol=.000001)

for alpha in alphas:
    ls = Lasso(alpha=alpha)
    ls.fit(X_train, y_train)
    errors.append(np.mean((ls.predict(X_test) - y_test) ** 2))

In [ ]:
plt.plot(alphas, errors)

In [ ]:
best_alpha = alphas[np.argmin(errors)]
best_alpha

In [ ]:
ls = Lasso(alpha=best_alpha, max_iter=100000, tol=.000001)
ls.fit(X_stand, y_7)

In [ ]:
# coefficient doesn't resemble model at all. but these have been scaled by
# their  std.  must divide by std
ls.coef_

In [ ]:
# that's better - very close to actual value of -1
ls.coef_ / X.std()

In [ ]:
# also look at intercept
ls.intercept_

# 9

In [ ]:
!ls

In [ ]:
college = pd.read_csv('data/college.csv')

In [ ]:
college['Private'].value_counts()

In [ ]:
college['private_yes'] = (college['Private'] == 'Yes') * 1

In [ ]:
X = college.iloc[:, 3:]

In [ ]:
y = college['Apps']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
# Error
np.mean((lr.predict(X_test) - y_test) ** 2)

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
X_std = X.iloc[:, :-1].std()

In [ ]:
X_std['private_yes'] = 1

In [ ]:
X_std

In [ ]:
rcv = RidgeCV(alphas=np.linspace(.01, 100, 1000), cv=10)
rcv.fit(X / X_std, y)